In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import matplotlib.pyplot as plt
import copy
import time

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
torch.__version__

import pandas as pd
import numpy as np
from PIL import Image
from skimage import io, transform

In [3]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

!unzip '/content/drive/My Drive/Vision/doors_imgs.zip'

train_data = pd.read_csv('/content/drive/My Drive/Vision/train/doors_train.csv')
test_data = pd.read_csv('/content/drive/My Drive/Vision/test/doors_test.csv')

Streaming output truncated to the last 5000 lines.
  inflating: doors_imgs/45804538-1.jpg  
  inflating: doors_imgs/45794571-2.jpg  
  inflating: doors_imgs/45805763-3.jpg  
  inflating: doors_imgs/45812932-2.jpg  
  inflating: doors_imgs/45696060-4.jpg  
  inflating: doors_imgs/45547091-4.jpg  
  inflating: doors_imgs/45809746-4.jpg  
  inflating: doors_imgs/45735677-3.jpg  
  inflating: doors_imgs/45730011-3.jpg  
  inflating: doors_imgs/45793569-6.jpg  
  inflating: doors_imgs/45805482-6.jpg  
  inflating: doors_imgs/45799413-2.jpg  
  inflating: doors_imgs/45640128-10.jpg  
  inflating: doors_imgs/45815659-3.jpg  
  inflating: doors_imgs/45803718-4.jpg  
  inflating: doors_imgs/45811791-5.jpg  
  inflating: doors_imgs/45780583-4.jpg  
  inflating: doors_imgs/45710805-2.jpg  
  inflating: doors_imgs/45785771-4.jpg  
  inflating: doors_imgs/45648781-3.jpg  
  inflating: doors_imgs/45719799-1.jpg  
  inflating: doors_imgs/45804164-8.jpg  
  inflating: doors_imgs/45766899-2.jpg  
  inf

In [25]:
Doors = np.array(train_data.Doors.unique())

class Dataset_Doors(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.doors = Doors

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        path = "/content/doors_imgs/{}".format(self.data.iloc[idx].img_path[1:].replace('/', '-'))
        image = transform.resize(io.imread(path), (224, 224))/255
        image = image[:, :, :3]
        
        y = np.where(self.doors == self.data.iloc[idx].Doors)[0]
        sample = (torch.Tensor(np.einsum('ijk->kij',image)), torch.Tensor(y).long())

        return sample

In [26]:
Doors

array(['4/5', '2/3', '>5'], dtype=object)

In [27]:
batch_size = 4
train = Dataset_Doors('/content/drive/My Drive/Vision/train/doors_train.csv')
val = Dataset_Doors('/content/drive/My Drive/Vision/test/doors_test.csv')

train_loader = DataLoader(train, batch_size=batch_size,shuffle=True, num_workers=4)
val_loader = DataLoader(val, batch_size=batch_size,shuffle=True, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_loader), 'val': len(val_loader)}

In [28]:
class model_init(nn.Module):
    def __init__(self):
        super(model_init, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(models.resnet18(pretrained=True))
        self.layers.append(nn.Linear(1000, 512)) 
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(512, 64))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(64, 3))
        self.layers.append(nn.Softmax())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [29]:
def train_helper(category, model, optimization, judge, timeout):
    if category == 'train':
        model.train()
    else:
        model.eval()

    loss_tmp = 0.0
    correct_tmp = 0

    for inputs, labels in dataloaders[category]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0]))
        optimization.zero_grad()

        with torch.set_grad_enabled(category == 'train'):
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = judge(outputs, labels.reshape((-1,)))

            if category == 'train':
                loss.backward()
                optimization.step()

        # statistics
        loss_tmp += loss.item() * inputs.size(0)
        correct_tmp += torch.sum(preds == labels.data)
    if category == 'train':
        timeout.step()

    epoch_loss = loss_tmp / dataset_sizes[category]
    epoch_acc = correct_tmp.double() / dataset_sizes[category]

    print('{} Loss: {:.4f} Accuracy: {:.4f}'.format(category, epoch_loss, epoch_acc/batch_size))
    
    return epoch_acc, copy.deepcopy(model.state_dict())

In [30]:
def train_model(model, judge, optimization, timeout, epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-----------------------------------------')
        train_helper('train', model, optimization, judge, timeout)
        accuracy_tmp, weight_tmp = train_helper('val', model, optimization, judge, timeout)
        if accuracy_tmp > best_accuracy:
            best_accuracy = accuracy_tmp
            best_model_weights = weight_tmp

        print()
        
    print('Best val Acc: {:4f}'.format(best_accuracy))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [31]:
criterion = nn.CrossEntropyLoss()
model = model_init()
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

In [32]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, epochs=7)

Epoch 1/7
-----------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


train Loss: 3.3276 Accuracy: 0.7127
val Loss: 3.1266 Accuracy: 0.7619

Epoch 2/7
-----------------------------------------
train Loss: 3.1185 Accuracy: 0.7651
val Loss: 2.9659 Accuracy: 0.8073

Epoch 3/7
-----------------------------------------
train Loss: 3.0103 Accuracy: 0.7939
val Loss: 2.9072 Accuracy: 0.8180

Epoch 4/7
-----------------------------------------
train Loss: 2.8385 Accuracy: 0.8397
val Loss: 2.8175 Accuracy: 0.8431

Epoch 5/7
-----------------------------------------
train Loss: 2.7989 Accuracy: 0.8493
val Loss: 2.8775 Accuracy: 0.8266

Epoch 6/7
-----------------------------------------
train Loss: 2.7910 Accuracy: 0.8516
val Loss: 2.8018 Accuracy: 0.8480

Epoch 7/7
-----------------------------------------
train Loss: 2.6980 Accuracy: 0.8753
val Loss: 2.7565 Accuracy: 0.8600

Best val Acc: 3.440000


In [33]:
torch.save(model.state_dict(), '/content/drive/My Drive/Vision/models/doors_model.pt')